# Caravan and Camping Club POI Generator 2021

In [1]:
# Install any missing libraries
!pip install simplekml
!pip install pandas
!pip install folium

In [2]:
# imports
import pandas as pd
import simplekml
import folium

In [4]:
df = pd.read_csv("data/Club-Sites.csv",names=['Longitude','Latitude','Address'], header=0, sep=',')
print(df.head(5))
print(df['Address'].head())

   Longitude   Latitude                                  Address
0  -1.902287  53.009451    "Alton T' Star ST10 3DW>01538 702219"
1  -1.649090  53.019150        "Ashbourne DE6 3EN>01335 370855""
2  -1.688239  53.169597        "Bakewell DE45 1NA>01629 636555""
3  -3.540269  52.934359            "Bala LL23 7LN>01678 530324""
4  -1.965871  54.551519  "Barnard Castle DL12 9DG>01833 630228""
0      "Alton T' Star ST10 3DW>01538 702219"
1          "Ashbourne DE6 3EN>01335 370855""
2          "Bakewell DE45 1NA>01629 636555""
3              "Bala LL23 7LN>01678 530324""
4    "Barnard Castle DL12 9DG>01833 630228""
Name: Address, dtype: object


### Extract Site Name, Telephone Number, and Address in to their own columns

In [5]:
df['SiteName'] = df['Address'].str.extract('([a-zA-Z].*(?=>))')

temp = df['Address'].apply(lambda x: pd.Series(str(x).split('>')))
print(temp)  # check if data is okay, one line had less quotes in it for some reason!!

# making separate phone number column from new data frame 
df["PhoneNumber"]= temp[1]

# Dropping old Address columns 
df.drop(columns =["Address"], inplace = True) 

# Remove extra quotes the camping club add to the end of each line!
df['PhoneNumber'] = df.apply(lambda x: x['PhoneNumber'][:-2], axis=1)

# df display 
df 

                             0               1
0      "Alton T' Star ST10 3DW   01538 702219"
1           "Ashbourne DE6 3EN  01335 370855""
2           "Bakewell DE45 1NA  01629 636555""
3               "Bala LL23 7LN  01678 530324""
4     "Barnard Castle DL12 9DG  01833 630228""
..                         ...             ...
97        "Winchcombe GL20 8NX  01242 620259""
98         "Windermere LA8 9JS  01539 821119""
99         "Wolverley DY10 3PX  01562 850909""
100     "Woodhall Spa LN10 6YU  01526 352911""
101  "Wyeside Campsite LD6 5LB  01597 810183""

[102 rows x 2 columns]


,Longitude,Latitude,SiteName,PhoneNumber
0,-1.902287,53.009451,Alton T' Star ST10 3DW,01538 70221
1,-1.649090,53.019150,Ashbourne DE6 3EN,01335 370855
2,-1.688239,53.169597,Bakewell DE45 1NA,01629 636555
3,-3.540269,52.934359,Bala LL23 7LN,01678 530324
4,-1.965871,54.551519,Barnard Castle DL12 9DG,01833 630228
...,...,...,...,...
97,-1.990634,51.990939,Winchcombe GL20 8NX,01242 620259
98,-2.802297,54.360939,Windermere LA8 9JS,01539 821119
99,-2.244030,52.408519,Wolverley DY10 3PX,01562 850909
100,-0.168271,53.152867,Woodhall Spa LN10 6YU,01526 352911


### Now create kml file for use by maps.me

### Once the .kml file has been created for maps.me will need to edit the file and change all instances of <styleUrl>#<number></styleUrl> with a valid maps.me styleUrl
    
<styleUrl>#placemark-blue</styleUrl>
<styleUrl>#placemark-red</styleUrl>
<styleUrl>#placemark-green</styleUrl>
<styleUrl>#placemark-yellow</styleUrl>
<styleUrl>#placemark-brown</styleUrl>
<styleUrl>#placemark-orange</styleUrl>
<styleUrl>#placemark-pink</styleUrl>    
<styleUrl>#placemark-purple</styleUrl>

In [6]:
kml = simplekml.Kml()
style = simplekml.Style()
style.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/paddle/orange-blank.png'
style.iconstyle.scale = 1

map = folium.Map(location=[54.0, -1.0], zoom_start=6)
for index, row in df.iterrows():
    #name = row['SiteName'] + ', ' + row['SiteAddress'] + ', ' + row['PhoneNumber']
    name = row['SiteName']
    address = row['PhoneNumber']
    phonenumber = row['PhoneNumber']
    coords = [(row['Longitude'], row['Latitude'])]
    pnt = kml.newpoint(name=name)
    pnt.coords = coords
    pnt.style = style
    pnt.phonenumber = phonenumber
    pnt.description = address
    
    popupText = folium.Html('<p>'+row['SiteName']+'</p><p>'+'</p><p>'+row['PhoneNumber']+'</p>', script=True)
    #popupText = folium.Html('<p>'+row['SiteName']+'</p><p>'+row['SiteAddress']+'</p>', script=True)
    popup = folium.Popup(popupText, max_width=300)
    folium.Marker([row['Latitude'], row['Longitude']],
                 popup=popup,
                 icon=folium.Icon(color='orange', icon='info-sign')).add_to(map)
    
kml.save(path="poi/ccc-sites-2021.kml")
map.save("poi/ccc-sites-2021.html")
map

In [46]:
#!pip list